# Likelihood nodal #

In [84]:
#Paquetes a cargar. Puede que sobre alguno o haya alguno redundante
import cobaya
import numpy as np
import math
from scipy.special import erf
from scipy.interpolate import CubicSpline
import camb
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [53]:
#Constantes cosmológicas:
c = 2.99792458E5;   HJPAS = 1/(c/100);

#Parámetros que no se van a samplear:
gamma = 0.545; OmegakJPAS = 0; AsJPAS = 2.09052E-9; nsJPAS = 0.9626; 

In [54]:
#Parámetros cosmológicos directos:
hJPASFid = 0.6736; OmegabJPASh2Fid = 0.02237; OmegaCDMJPASh2Fid = 0.1200; 
OmegamJPASFid = 0.3153;

#Parámetros cosmológicos indirectos:
OmegabJPASFid = OmegabJPASh2Fid/hJPASFid**2; OmegaCDMJPASFid = OmegaCDMJPASh2Fid/hJPASFid**2;
OmegaLJPASFid = 1 - OmegamJPASFid;

#Parámetros cosmológicos fuera del fiducial:
hJPAS = hJPASFid + hJPASFid/100;
OmegabJPASh2 = OmegabJPASh2Fid + OmegabJPASh2Fid/100;
OmegaCDMJPASh2 = OmegaCDMJPASh2Fid + OmegaCDMJPASh2Fid/100; 
OmegamJPAS = OmegamJPASFid + OmegamJPASFid/100;

#Parámetros cosmológicos indirectos fuera del fiducial:
OmegabJPAS = OmegabJPASh2/hJPAS**2; OmegaCDMJPAS = OmegaCDMJPASh2/hJPAS**2;
OmegaLJPAS = 1 - OmegamJPAS;

In [85]:
#Bineado de k y de z
#Límites y pasos de los arrays. Escalas en unidades de h.
kminKArrayCompleto = 0.001;   kmaxKArrayCompleto = 2.4900;  pasoKArrayCompleto = 0.025;
zmin = 1.7;   zmax = 2.9;   pasoz = 0.2;

#Bines de k, completos y reducidos
KArrayCompleto = np.exp(np.arange(math.log(kminKArrayCompleto), math.log(kmaxKArrayCompleto), pasoKArrayCompleto) )
KArray = KArrayCompleto[range(121,246)]

#Bines de z:
za = np.arange(zmin, zmax+pasoz/2, pasoz)

In [97]:
# Define a class to read the data
def read_data(path_to_data):
    data = {}

    Simulated_pk_filename = path_to_data+'FicticioHighZArrayEnK.dat'
    Simulated_densities = path_to_data+'DensityHighZ.dat'


    data['pkz'] = np.zeros((len(za), len(KArray)))
    data['ndz'] = np.zeros(len(za))
  

    with open(Simulated_pk_filename) as file:
        for i in range(len(KArray)):
            line = file.readline().split()
            data['pkz'][0][i] = float(line[2])
            data['pkz'][1][i] = float(line[3])
            data['pkz'][2][i] = float(line[4])
            data['pkz'][3][i] = float(line[5])
            data['pkz'][4][i] = float(line[6])
            data['pkz'][5][i] = float(line[7])
            data['pkz'][6][i] = float(line[8])

    with open(Simulated_densities) as file:
        for i in range(len(za)):
            line = file.readline().split()
            data['ndz'][i] = float(line[1])

    return data

In [98]:
# Aquí se da el string del path para ejecutar read_data
# Read data as dictionary
data = read_data('/Users/guillermo/Desktop/')
# show data, we do not need Dz, fz, Haz, these are requested by camb
data.keys()

dict_keys(['pkz', 'ndz'])

In [48]:
# Let's try to reproduce with simple CAMB the nodes, Fig 1 of the paper.

#Se dan los valores de los nodos

nodes_log_k = [-5, -3.3]
nodes_log_PPS = [3.5, 3.7]

#Se deshace la escala log
nodes_k = np.exp(nodes_log_k)
nodes_PPS =np.exp(nodes_log_PPS)*1e-10

# Let's compare to scipy
#Aquí se realiza algún tipo de interpolación
func = interp1d(nodes_k, nodes_PPS,
                axis=0,  # interpolate along columns
                bounds_error=False,
                kind='quadratic',
                fill_value=(nodes_PPS[0], nodes_PPS[-1]))

#Set up a new set of parameters for CAMB
#Metemos en pars los parámetros de CAMB, y en la subcategoría de InitPower esos datos. 
pars = camb.CAMBparams()
pars.InitPower = camb.initialpower.SplinedInitialPower()

#This function sets up with one massive neutrino and helium set using BBN consistency
#Aquí cambiamos los parámetros por defecto de CAMB por estos que nos interesan a nosotros
pars.set_cosmology(H0=hJPAS*100, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0.0, omk=OmegakJPAS, tau=0.06)

#Aquí decimos que en esta subcategoría se incorporen los ks de los nodos con su interpolación
#pars.InitPower.set_scalar_table(nodes_k, nodes_PPS)
pars.InitPower.set_scalar_log_regular(KArray[0], KArray[-1], func(KArray))

#Ahora damos los redshifts de nuestro catálogo con el kmax del mismo para el espectro de materia.
pars.set_matter_power(redshifts=za, kmax=KArrayCompleto[-1])

Note: redshifts have been re-sorted (earliest first)


class: <CAMBparams>
 WantCls = True
 WantTransfer = True
 WantScalars = True
 WantTensors = False
 WantVectors = False
 WantDerivedParameters = True
 Want_cl_2D_array = True
 Want_CMB = True
 Want_CMB_lensing = True
 DoLensing = True
 NonLinear = NonLinear_none
 Transfer: <TransferParams>
   high_precision = True
   accurate_massive_neutrinos = False
   kmax = 2.44060197762477
   k_per_logint = 0
   PK_num_redshifts = 7
   PK_redshifts = [2.8999999999999995, 2.6999999999999997, 2.5, 2.3, 2.0999999999999996, 1.9, 1.7]
 want_zstar = False
 want_zdrag = False
 min_l = 2
 max_l = 2500
 max_l_tensor = 600
 max_eta_k = 5000.0
 max_eta_k_tensor = 1200.0
 ombh2 = 0.022
 omch2 = 0.122
 omk = 0.0
 omnuh2 = 0.000644866570625114
 H0 = 67.5
 TCMB = 2.7255
 YHe = 0.24569492503497048
 num_nu_massless = 2.0293333333333337
 num_nu_massive = 1
 nu_mass_eigenstates = 1
 share_delta_neff = False
 nu_mass_degeneracies = [1.0146666666666666]
 nu_mass_fractions = [1.0]
 nu_mass_numbers = [1]
 InitPower: <Spl

In [55]:
# get camb results 
# Obtenemos los resultados de CAMB y los comparamos a nuestros nodos interpolados

# Obtenemos los resultados de CAMB con todos los cambios que hemos hecho en 'pars':
results = camb.get_results(pars)

#Creamos los ejes x e y para representar. Array de ks, llamado 'k':
kplot = 10**np.linspace(-5, 1, 500)

#Valores del P(k) en ese array 'k':
scalar_pk = pars.scalar_power(kplot)


In [56]:
from camb import model #Se importa un modelo dentro de CAMB?

#Linear spectra
pars.NonLinear = model.NonLinear_none #Esto pone las no-linearidades a 0
results = camb.get_results(pars) 

#Se da valor a estas variables desde el P materia(k) de CAMB, especificando nuestras escalas extremas y el número de puntos
#kh son las ks, pk el valor del Pmateria(k)
kh, z, pk = results.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

#Lo mismo con sigma8 para cada bin de z
s8 = np.array(results.get_sigma8())
print(z)

[1.7, 1.9, 2.0999999999999996, 2.3, 2.5, 2.6999999999999997, 2.8999999999999995]


In [62]:
# I assume this method above is ok, so I will now create the classes to interface with Cobaya
# I will create a cobaya theory NodesInPrimordialPk and a cobaya external likelihood Pklike classes

#Se crean las clases para interactuar con Cobaya: NodesInPrimordialPk (teoría) y Pklike (likelihood)

from cobaya.theory import Theory
from cobaya.likelihood import Likelihood

In [68]:
#Clase con la teoría, es decir, con la modificación del Primordial para incluir nuestros nodos
class NodesInPrimordialPk(Theory):

    def initialize(self): #Creamos una función que devolverá self. Ahí metemos unos ks.
        # need to provide valid results at wide k range, any that might be used, please change accordantly
        self.ks = KArray

    def calculate(self, state, want_derived=True, **params_values_dict): #Esta función hace....
        pivot_scalar = 0.05 #please change if you need
        #Ahora metemos en k1, pk1... los valores de los nodos anteriormente fijados
        nodes_k = [params_values_dict['k1'], params_values_dict['k2']]
        nodes_PPS = [params_values_dict['pk1'], params_values_dict['pk2']]
        
        #Se interpolan estos nodos.
        Pk_func = interp1d(nodes_k, nodes_PPS,
                axis=0,  # interpolate along columns
                bounds_error=False,
                kind='quadratic',
                fill_value=(nodes_PPS[0], nodes_PPS[-1]))
        
        #Parece que aquí definimos la variable state, en la que construimos el P(k) en todas las escalas
        state['primordial_scalar_pk'] = {'kmin': self.ks[0], 'kmax': self.ks[-1],
                                         'Pk': Pk_func(self.ks), 'log_regular': True}

    #No entiendo para que sirven estas dos funciones:
    def get_primordial_scalar_pk(self):
        return self.current_state['primordial_scalar_pk']
   
    #Parece que aquí hay que indicar qué parámetros nodales usamos
    def get_can_support_params(self):
        # Please, define here as many node params you like
        return ['k1', 'k2', 'pk1', 'pk2']


In [99]:
#Clase con el likelihood. Aquí tendremos que introducir nuestro modelo y el cálculo del likelihood

class Pklike(Likelihood): #Se define la clase.
    
    def initialize(self):  # Función initialize, que crea un bineado de redshift basado en read data

        # Se leen los datos de read_data, como arriba
        # Plese, be aware that you need to change the path
        self.data = read_data('/Users/guillermo/Desktop/')

        # Se da un grid de zs con extremos en nuestros bines y 150 pasos
        # If you need some quantities at z = 0 you need to have z_win also at zero, please change accordantly
        self.z_win = za

   
        # No entiendo para qué sirve esta función. Parece que evalua funciones comológicas en nuestro bineado de z z_win
    
    def get_requirements(self):
        
        return {'omegam': None,                 #¿Por qué incluir valores en los que no se evalúa nada en z_win?
                'Pk_interpolator': {'z': self.z_win, 'k_max': 10, 'nonlinear': False, 'vars_pairs': ([['delta_tot', 'delta_tot']])},
                'comoving_radial_distance': {'z': self.z_win},
                'angular_diameter_distance': {'z': self.z_win},
                'Hubble': {'z': self.z_win, 'units': 'km/s/Mpc'},
                'sigma8_z': {'z': self.z_win},
                 #'fsigma8': {'z': self.z_win, 'units': None},
                'CAMBdata': None}

    # Esta función parece definir el monopolo:
    
    def monopole(self, **params_dic):

        results = self.provider.get_CAMBdata()   #Parece que esto lee los datos de CAMB
        
        #Aquí defino el bineado en k
        kminKArrayCompleto = 0.001;   kmaxKArrayCompleto = 2.4900;  pasoKArrayCompleto = 0.025;
        KArrayCompleto = np.exp( np.arange(math.log(kminKArrayCompleto), math.log(kmaxKArrayCompleto), pasoKArrayCompleto) )
        KArray = KArrayCompleto[range(121,246)]
        
        # Aquí se puede acceder al PPS
        ks = KArray
        pps = results.Params.scalar_power(ks)       
        
        #Aquí creamos todas las funciones y variables necesarias para generar el P Kaiser.
        Omegam = self.provider.get_Omega('baryon')+self.provider.get_Omega('cdm')
        OmegamFid = self.providerFid.get_Omega('baryon')+self.providerFid.get_Omega('cdm')
        Ez = np.sqrt( Omegam*(1+self.z_win)**3+(1-Omegam) );    EzFid = np.sqrt( OmegamFid*(1+self.z_win)**3+(1-OmegamFid) )
        H = HJPAS * Ez
        f = (Omegam*(1+self.z_win)**3*1/(Ez**2))**gamma
        sigma8z0 = self.providerz0.get_sigma8_z(self.z_winz0)
        DeReves = self.provider.get_sigmaR_z(8,self.z_win)/self.providerz0.get_sigmaR_z(8,self.z_winz0)
        De = DeReves[::-1]
        def bJPAS(z):
          return 0.53+0.289*(1+z)**2
        A = De*bJPAS(za)*sigma8z0
        R = De*f*sigma8z0

        #Fotometría
        DeltazJPAS = 0.00364236313918151
        sigmar = DeltazJPAS*(1+self.z_win)/H

        # This is the matter power spectrum interpolator:
        pk = self.provider.get_Pk_interpolator(('delta_tot', 'delta_tot'), nonlinear=False)   #Parece que aquí se obtiene el Pmateria
        pk_delta = pk.P(ks, self.z_win)         # pk_delta is an array de pmateria evaluado en ks and zs

        
        #Fingers of God
        sigmap = (1/(6*np.pi**2)*(De/1)**2*integrate.quad(lambda k:  pk.P(k, self.z_win[0]) , ks[0], ks[len(ks)-1])[0])**0.5
        def FFog(mu,k):
            return 1/(1+(f[0]*k*mu*sigmap[0])**2)

        
        #Efecto AP
        Xi = self.provider.get_comoving_radial_distance(self.z_win)*hJPAS;   XiFid = self.providerFid.get_comoving_radial_distance(self.z_win)*hJPAS;
        DA = Xi/(1+self.z_win);   DAFid = XiFid/(1+self.z_win)
        FactorAP = DAFid**2*Ez/( DA**2*EzFid )

        def Q(mu):
            return ((Ez[0]**2*Xi[0]**2*mu**2-EzFid[0]**2*XiFid[0]**2*(mu**2-1))**0.5/(EzFid[0]*Xi[0]))
            
        def muObs(mu):
            return mu*Ez[0]/(EzFid[0]*Q(mu))
           
        def kObs(mu,k):
            return Q(mu)*k 

        #P de galaxias final
        def Pg(mu,k):
            return FactorAP[0]*FFog(muObs(mu),kObs(mu,k))*(A[0]+R[0]*muObs(mu)**2)**2 * PmatInterCAMB(kObs(mu,k))/sigma8z0**2 *np.exp(-(k*mu*sigmar[0])**2)

        #Se usa la regla del trapecio con 2000 pasos
        def Pgmonopole(k):
            mu = np.arange(-1, 1, 1/1000)
            return 1/2 * integrate.trapz(Pg(mu, k), mu)
            
        PgmonopoleValores = np.zeros(len(ks))
        for i in range(0, len(ks)):
             PgmonopoleValores[i] = Pgmonopole(ks[i])

        #Covarianza
        
        #Importación y lectura de las densidades
        ImportacionDensityHighZ = [i.strip().split() for i in open("/Users/guillermo/Desktop/DensityHighZ.dat").readlines()]

        DensityHighZ = np.zeros(len(ImportacionDensityHighZ));

        for i in range(0, len(ImportacionDensityHighZ)):
          DensityHighZ[i] = ImportacionDensityHighZ[i][1]

        #Definición del volumen (requiere distancia angular con unidades y binear en zupper y zlower)
        #Área del cielo
        fsky = 0.2575;

        #Bines de z por arriba y por abajo:
        z_win_upper = self.z_win+(self.z_win[[1]]-self.z_win[[0]])/2;    z_win_lower = self.z_win-(self.z_win[[1]]-self.z_win[[0]])/2;

        #Distancia angular para los bines z upper y lower:
        XiZaLower = resultszlower.comoving_radial_distance(z_win_lower)*hJPAS
        XiZaUpper = resultszupper.comoving_radial_distance(z_win_upper)*hJPAS

        #Definición de volumen:
        Vol = 4*np.pi*fsky/3*(XiZaUpper**3-XiZaLower**3)

        #Definición del número de modos (requiere arrays en kupper y klower):

        #Bines de k por arriba y por abajo
        KArrayUpper = np.zeros(len(ks)); KArrayLower = np.zeros(len(ks));

        for i in range(0, len(ks)-1):
            KArrayUpper[i] = ks[i] + (ks[i+1]-ks[i])/2;   KArrayLower[i] = ks[i] - (ks[i+1]-ks[i])/2;

        KArrayUpper[-1] = KArrayUpper[-2];  KArrayLower[-1] = KArrayLower[-2];

        #Número de modos. Depende de las variables k1 y k2, que debe corresponderse a kupper y klower
        def Nk(k1,k2):
            return Vol[0] * (4*np.pi/3*(k1**3-k2**3))/((2*np.pi)**3)

        #Evaluamos Nk para cada valor de nuestro array de k
        NkEvaluado = np.zeros(len(ks))
        for i in range(0, len(ks)):
            NkEvaluado[i] = Nk(KArrayUpper[i],KArrayLower[i])

        #Definición de la covarianza

        #Va a depender de k1 y k2. No me gusta mucho:
        def Cov(k,k1,k2):
            return 2 * (Pgmonopole(k) + 1/DensityHighZ[0])**2 / Nk(k1,k2)

        #Evaluamos Cov para nuestros k
        CovEvaluado = 2 *(PgmonopoleValores + 1/DensityHighZ[0])**2 / NkEvaluado

        # Other cosmological quantities:
        angular_distance = self.provider.get_angular_diameter_distance(self.z_win),
        H = self.provider.get_Hubble(self.z_win),
        fsigma8 = self.provider.get_fsigma8(self.z_win)
        sigma8 = self.provider.get_sigma8_z(self.z_win)


    # Esta función computa el loglikelihood
    
    def logp(self, **params_values):       
        
        # Calcular el monopolo:
        
        Pk = self.monopole(**params_values)    #Aquí se llama a la función monopolo
        
        # Calcular el loglikelihood

        return 0


In [83]:
# This is how you pass input to Cobaya
# Keep in mind that right know the parameters are hardcoded (fijos), you need to change to priors
# Diccionario que le pasamos a Cobaya, donde linkamos con nuestros códigos de teoróa y el likelihood.

info = {'debug': True,                        #Esto permite obtener info de los errores
        'likelihood': {'jpass': Pklike},      #Aquí se engancha el likelihood (nombre jpass) que hemos definido en la clase de arriba
        'theory': {'camb': {"external_primordial_pk": True},
                   'my_pk': NodesInPrimordialPk},      #Aquí le pasamos nuestra clase de teoría, con nombre "my_pk".
       'params': {
        # Parámetros cosmológicos fijados
        'tau': 0.07, 'mnu': 0.00, 'nnu': 3.046,
        # Parámetros nodales, flat priors
        'P1': {'prior': {'min': 1e-9, 'max': 4e-9}, 'latex': 'P_1'},
        'P2': {'prior': {'min': 1e-9, 'max': 4e-9}, 'latex': 'P_2'},
        # Parámetros cosmológicos a samplear. Loc y scale son, en prior gaussiano, valor medio y desviación estandar.
        'ombh2': {'prior': {'dist': 'norm', 'loc': OmegabJPASh2Fid, 'scale': 0.00015}, 'latex': 'Omega_bh^2'},
        'omch2': {'prior': {'dist': 'norm', 'loc': OmegaCDMJPASFid, 'scale': 0.0012}, 'latex': 'Omega_ch^2'},
        'H0': {'prior': {'dist': 'norm', 'loc': hJPASFid*100, 'scale': 0.54}, 'latex': 'H_0'}}}

In [ ]:
# Let's reproduce the same matter power spectrum as the one by single camb through the cobaya interface
# Se reproduce el Pmateria construido por CAMB con la interfaz de Cobaya:

from cobaya.model import get_model     
model = get_model(info)          #Se construye un modelo con el diccionario info

model.logposterior({}) 

camb_results = model.provider.get_CAMBdata();

pk = model.provider.get_Pk_interpolator(('delta_tot', 'delta_tot'), nonlinear=False)
